In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
import numpy as np
import qiskit
%matplotlib inline
import random

In [2]:
# Step0

n=8  # length of message = n, n must be multiple of 4.

def random_01str(list,length):
    for i in range(length):
        list.append(random.randint(0,1))
        
message = []
message.clear()
random_01str(message,n)
print("message : {}".format(message))

message : [1, 0, 0, 1, 1, 1, 0, 1]


In [3]:
def XOR_message(list1,list2):
    list_part1=[]
    list_part1.clear()
    list_part2=[]
    list_part2.clear()
    for i in range(len(list1)//2):
        list_part1.append(list1[i])
        list_part2.append(list1[i+(len(list1)//2)])
        
    print("message_part1 : {}".format(list_part1))
    print("message_part2 : {}".format(list_part2))
        
    XOR_list=[]
    XOR_list.clear()
    for j in range(len(list1)//2):
        if list_part1[j]==list_part2[j]:
            XOR_list.append(0)
        else:
            XOR_list.append(1)
    
    print("XOR_message : {}".format(XOR_list))
    list2.extend(XOR_list)    
        
        

XOR_m=[]  
XOR_m.clear()

XOR_message(message,XOR_m)
print("XOR_m : {}".format(XOR_m))

message_part1 : [1, 0, 0, 1]
message_part2 : [1, 1, 0, 1]
XOR_message : [0, 1, 0, 0]
XOR_m : [0, 1, 0, 0]


In [4]:
m_XOR_m=[]  
m_XOR_m.clear()
m_XOR_m=message+XOR_m
print("m_XOR_m : {}".format(m_XOR_m))

m_XOR_m : [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0]


In [5]:
preshared_string=[]  
preshared_string.clear()

def random_012str(list1,length):
    for i in range(length):
        list1.append(random.randint(0,2))
        
random_012str(preshared_string,(n*3)//4)
print("preshared_string : {}".format(preshared_string))

preshared_string : [2, 2, 1, 2, 2, 1]


In [6]:
# Step1 : depends on m||XOR(m), operate N |0> qubits with H(0) and H,Y(1)
print("m_XOR_m ,I(0) or Y(1): {}".format(m_XOR_m))

# generate one qubit

q = QuantumRegister(1,'q')
c = ClassicalRegister(2,'c')

# Alice
m_circ0 = QuantumCircuit(q,c)  # |+>
m_circ0.h(q[0])  
#m_circ0.barrier(q)

m_circ1 = QuantumCircuit(q,c)  # |->
m_circ1.x(q[0])
m_circ1.h(q[0])
#m_circ1.barrier(q)


#m_circ0.draw()
#m_circ1.draw()

m_XOR_m ,I(0) or Y(1): [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0]


In [7]:
# generate circuits

m_circ=[m_circ0,m_circ1]

circuits_m_XOR_m=[]
circuits_m_XOR_m.clear()

for i in range((n*3)//2):
    circuits_m_XOR_m.append(m_circ[m_XOR_m[i]])
    
print("length of circuits_m_XOR_m : {}".format(len(circuits_m_XOR_m)))
circuits_m_XOR_m[0].draw()

length of circuits_m_XOR_m : 12


In [8]:
# Step2 : depends on Pre-shared String, insert decoy photons (random qubits) for each two m_XOR_m qubits
#         0 --> insert in the middle, 1 --> insert at the left side, 2 --> insert at the right side
print("preshared_string : {}".format(preshared_string))

#decoy_01=[]
#decoy_01.clear()
#random_01str(decoy_01,(n*3)//4)
#print("decoy_01 ,0(0) or 1(1) : {}".format(decoy_01))

#decoy_ZX=[]
#decoy_ZX.clear()
#random_01str(decoy_ZX,(n*3)//4)
#print("decoy_ZX ,Z(0) or X(1) : {}".format(decoy_ZX))

decoy=[]
decoy.clear()
random_01str(decoy,(n*3)//4)
print("decoy ,0(|+>) or 1(|->) : {}".format(decoy))


Decoy_circ2 = QuantumCircuit(q,c)  # |+>
Decoy_circ2.h(q[0])  
#Decoy_circ2.barrier(q)

Decoy_circ3 = QuantumCircuit(q,c)  # |->
Decoy_circ3.x(q[0])
Decoy_circ3.h(q[0])
#Decoy_circ3.barrier(q)


#Decoy_circ2.draw()
#Decoy_circ3.draw()

preshared_string : [2, 2, 1, 2, 2, 1]
decoy ,0(|+>) or 1(|->) : [0, 0, 0, 0, 0, 0]


In [9]:
# generate circuits

Decoy_circ=[Decoy_circ2,Decoy_circ3]

circuits_Decoy=[]
circuits_Decoy.clear()

for i in range((n*3)//4):
    circuits_Decoy.append(Decoy_circ[decoy[i]])
    
print("length of circuits_Decoy : {}".format(len(circuits_Decoy)))
circuits_Decoy[0].draw()

length of circuits_Decoy : 6


In [10]:
circuits_MD_step2=[]
circuits_MD_step2.clear()

for i in range((n*3)//4):
    if preshared_string[i]==0:  # mdm
        circuits_MD_step2.append(circuits_m_XOR_m[2*i])
        circuits_MD_step2.append(circuits_Decoy[i])
        circuits_MD_step2.append(circuits_m_XOR_m[2*i+1])
    elif preshared_string[i]==1:  #dmm
        circuits_MD_step2.append(circuits_Decoy[i])
        circuits_MD_step2.append(circuits_m_XOR_m[2*i])
        circuits_MD_step2.append(circuits_m_XOR_m[2*i+1])
    elif preshared_string[i]==2:  #mmd
        circuits_MD_step2.append(circuits_m_XOR_m[2*i])
        circuits_MD_step2.append(circuits_m_XOR_m[2*i+1])
        circuits_MD_step2.append(circuits_Decoy[i])
    

    
print("length of circuits_MD_step2 : {}".format(len(circuits_MD_step2)))
circuits_MD_step2[0].draw()

length of circuits_MD_step2 : 18


In [11]:
# Step3 : Alice_lock

def random_lock(list1,length):
    for i in range(length):
        list1.append(random.randint(0,360))
    
Alice_lock_rx=[]  
Alice_lock_rx.clear()    
random_lock(Alice_lock_rx,((n*3)//4)+((n*3)//2))

Alice_lock_ry=[]  
Alice_lock_ry.clear()    
random_lock(Alice_lock_ry,((n*3)//4)+((n*3)//2))

Alice_lock_rz=[]  
Alice_lock_rz.clear()    
random_lock(Alice_lock_rz,((n*3)//4)+((n*3)//2))

print("Alice_lock_rx : {}".format(Alice_lock_rx))
#print("length of Alice_lock_rx : {}".format(len(Alice_lock_rx)))
print("Alice_lock_ry : {}".format(Alice_lock_ry))
#print("length of Alice_lock_ry : {}".format(len(Alice_lock_ry)))
print("Alice_lock_rz : {}".format(Alice_lock_rz))
#print("length of Alice_lock_rz : {}".format(len(Alice_lock_rz)))

Alice_lock_rx : [247, 228, 109, 325, 296, 157, 160, 233, 285, 134, 87, 122, 18, 51, 80, 17, 261, 219]
Alice_lock_ry : [104, 297, 122, 201, 80, 138, 290, 66, 200, 55, 255, 255, 349, 89, 59, 345, 215, 347]
Alice_lock_rz : [152, 205, 114, 110, 198, 187, 27, 32, 359, 285, 156, 266, 208, 269, 325, 162, 184, 29]


In [12]:
circuits_Alice_lock_rx=[]
circuits_Alice_lock_rx.clear()

for i in range(((n*3)//4)+((n*3)//2)):
    Rx_circ = QuantumCircuit(q,c) 
    Rx_circ.rx(2*np.pi*Alice_lock_rx[i]/360,q[0])
    #Rx_circ.barrier(q)
    circuits_Alice_lock_rx.append(Rx_circ)
    
print("length of circuits_Alice_lock_rx : {}".format(len(circuits_Alice_lock_rx)))
circuits_Alice_lock_rx[0].draw()

length of circuits_Alice_lock_rx : 18


In [13]:
circuits_Alice_lock_ry=[]
circuits_Alice_lock_ry.clear()

for i in range(((n*3)//4)+((n*3)//2)):
    Ry_circ = QuantumCircuit(q,c) 
    Ry_circ.ry(2*np.pi*Alice_lock_ry[i]/360,q[0])
    #Ry_circ.barrier(q)
    circuits_Alice_lock_ry.append(Ry_circ)
    
print("length of circuits_Alice_lock_ry : {}".format(len(circuits_Alice_lock_ry)))
circuits_Alice_lock_ry[0].draw()

length of circuits_Alice_lock_ry : 18


In [14]:
circuits_Alice_lock_rz=[]
circuits_Alice_lock_rz.clear()

for i in range(((n*3)//4)+((n*3)//2)):
    Rz_circ = QuantumCircuit(q,c) 
    Rz_circ.rz(2*np.pi*Alice_lock_rz[i]/360,q[0])
    #Rz_circ.barrier(q)
    circuits_Alice_lock_rz.append(Rz_circ)
    
print("length of circuits_Alice_lock_rz : {}".format(len(circuits_Alice_lock_rz)))
circuits_Alice_lock_rz[0].draw()

length of circuits_Alice_lock_rz : 18


In [15]:
circuits_Alice_lock=[]
circuits_Alice_lock.clear()

for i in range(((n*3)//4)+((n*3)//2)):
    circuits_Alice_lock.append(circuits_Alice_lock_rx[i]+circuits_Alice_lock_ry[i]+circuits_Alice_lock_rz[i])
    
print("length of circuits_Alice_lock : {}".format(len(circuits_Alice_lock)))
circuits_Alice_lock[0].draw()

length of circuits_Alice_lock : 18


In [16]:
circuits_MD_step3=[]
circuits_MD_step3.clear()

for i in range(((n*3)//4)+((n*3)//2)):
    circuits_MD_step3.append(circuits_MD_step2[i]+circuits_Alice_lock[i])
    

print("length of circuits_MD_step3 : {}".format(len(circuits_MD_step3)))
circuits_MD_step3[0].draw()

length of circuits_MD_step3 : 18


In [17]:
# Step4 : Send to Bob and Bob operates M with Bob_lock

Bob_lock_rx=[]  
Bob_lock_rx.clear()    
random_lock(Bob_lock_rx,((n*3)//2))

Bob_lock_ry=[]  
Bob_lock_ry.clear()    
random_lock(Bob_lock_ry,((n*3)//2))

Bob_lock_rz=[]  
Bob_lock_rz.clear()    
random_lock(Bob_lock_rz,((n*3)//2))

print("Bob_lock_rx : {}".format(Bob_lock_rx))
#print("length of Bob_lock_rx : {}".format(len(Bob_lock_rx)))
print("Bob_lock_ry : {}".format(Bob_lock_ry))
#print("length of Bob_lock_ry : {}".format(len(Bob_lock_ry)))
print("Bob_lock_rz : {}".format(Bob_lock_rz))
#print("length of Bob_lock_rz : {}".format(len(Bob_lock_rz)))

Bob_lock_rx : [169, 185, 72, 164, 303, 303, 254, 60, 289, 247, 187, 138]
Bob_lock_ry : [79, 356, 262, 55, 97, 90, 283, 13, 6, 315, 354, 140]
Bob_lock_rz : [89, 93, 186, 30, 206, 155, 49, 47, 289, 19, 228, 230]


In [18]:
circuits_Bob_lock=[]
circuits_Bob_lock.clear()

for i in range(((n*3)//2)):
    Rx_circ = QuantumCircuit(q,c) 
    Rx_circ.rx(2*np.pi*Bob_lock_rx[i]/360,q[0])
    Ry_circ = QuantumCircuit(q,c) 
    Ry_circ.ry(2*np.pi*Bob_lock_ry[i]/360,q[0])
    Rz_circ = QuantumCircuit(q,c) 
    Rz_circ.rz(2*np.pi*Bob_lock_rz[i]/360,q[0])
    #Rz_circ.barrier(q)
    circuits_Bob_lock.append(Rx_circ+Ry_circ+Rz_circ)
    
print("length of circuits_Bob_lock : {}".format(len(circuits_Bob_lock)))
circuits_Bob_lock[0].draw()

length of circuits_Bob_lock : 12


In [19]:
# not lock decoy

I_circ = QuantumCircuit(q,c)  # Identity
I_circ.iden(q[0])
#I_circ.barrier(q)
#I_circ.draw()


circuits_I_and_Bob_lock=[]
circuits_I_and_Bob_lock.clear()

for i in range((n*3)//4):
    if preshared_string[i]==0:  # mdm
        circuits_I_and_Bob_lock.append(circuits_Bob_lock[2*i])
        circuits_I_and_Bob_lock.append(I_circ)
        circuits_I_and_Bob_lock.append(circuits_Bob_lock[2*i+1])
    elif preshared_string[i]==1:  #dmm
        circuits_I_and_Bob_lock.append(I_circ)
        circuits_I_and_Bob_lock.append(circuits_Bob_lock[2*i])
        circuits_I_and_Bob_lock.append(circuits_Bob_lock[2*i+1])
    elif preshared_string[i]==2:  #mmd
        circuits_I_and_Bob_lock.append(circuits_Bob_lock[2*i])
        circuits_I_and_Bob_lock.append(circuits_Bob_lock[2*i+1])
        circuits_I_and_Bob_lock.append(I_circ)
    

    
print("length of circuits_I_and_Bob_lock : {}".format(len(circuits_I_and_Bob_lock)))
circuits_I_and_Bob_lock[0].draw()

length of circuits_I_and_Bob_lock : 18


In [20]:
circuits_MD_step4=[]
circuits_MD_step4.clear()

for i in range(((n*3)//4)+((n*3)//2)):
        circuits_MD_step4.append(circuits_MD_step3[i]+circuits_I_and_Bob_lock[i])
    

print("length of circuits_MD_step4 : {}".format(len(circuits_MD_step4)))
circuits_MD_step4[1].draw()

length of circuits_MD_step4 : 18


In [21]:
# Step5 : Send to Alice, and Alice checks decoy

circuits_Alice_check_decoy=[]
circuits_Alice_check_decoy.clear()

circuits_Alice_m_after_check=[]
circuits_Alice_m_after_check.clear()

for i in range((n*3)//4):
    if preshared_string[i]==0:  # mdm
        circuits_Alice_m_after_check.append(circuits_MD_step4[3*i])
        circuits_Alice_check_decoy.append(circuits_MD_step4[3*i+1])
        circuits_Alice_m_after_check.append(circuits_MD_step4[3*i+2])
    elif preshared_string[i]==1:  #dmm
        circuits_Alice_check_decoy.append(circuits_MD_step4[3*i])
        circuits_Alice_m_after_check.append(circuits_MD_step4[3*i+1])
        circuits_Alice_m_after_check.append(circuits_MD_step4[3*i+2])
    elif preshared_string[i]==2:  #mmd
        circuits_Alice_m_after_check.append(circuits_MD_step4[3*i])
        circuits_Alice_m_after_check.append(circuits_MD_step4[3*i+1])
        circuits_Alice_check_decoy.append(circuits_MD_step4[3*i+2])
    

    
print("length of circuits_Alice_check_decoy : {}".format(len(circuits_Alice_check_decoy)))
print("length of circuits_Alice_m_after_check : {}".format(len(circuits_Alice_m_after_check)))

circuits_Alice_check_decoy[0].draw()
#circuits_Alice_m_after_check[0].draw()

length of circuits_Alice_check_decoy : 6
length of circuits_Alice_m_after_check : 12


In [31]:
Alice_unlock_rx=[]  
Alice_unlock_rx.clear()    

Alice_unlock_ry=[]  
Alice_unlock_ry.clear()    

Alice_unlock_rz=[]  
Alice_unlock_rz.clear()    

for i in range(((n*3)//4)+((n*3)//2)):
    Alice_unlock_rx.append(360-Alice_lock_rx[i]) 
    Alice_unlock_ry.append(360-Alice_lock_ry[i]) 
    Alice_unlock_rz.append(360-Alice_lock_rz[i]) 
       
    
print("Alice_unlock_rx : {}".format(Alice_unlock_rx))
print("Alice_unlock_ry : {}".format(Alice_unlock_ry))
print("Alice_unlock_rz : {}".format(Alice_unlock_rz))
print("length of Alice_unlock_rx : {}".format(len(Alice_unlock_rx)))
print("length of Alice_unlock_ry : {}".format(len(Alice_unlock_ry)))
print("length of Alice_unlock_rz : {}".format(len(Alice_unlock_rz)))

Alice_unlock_rx : [113, 132, 251, 35, 64, 203, 200, 127, 75, 226, 273, 238, 342, 309, 280, 343, 99, 141]
Alice_unlock_ry : [256, 63, 238, 159, 280, 222, 70, 294, 160, 305, 105, 105, 11, 271, 301, 15, 145, 13]
Alice_unlock_rz : [208, 155, 246, 250, 162, 173, 333, 328, 1, 75, 204, 94, 152, 91, 35, 198, 176, 331]
length of Alice_unlock_rx : 18
length of Alice_unlock_ry : 18
length of Alice_unlock_rz : 18


In [37]:
circuits_Alice_unlock=[]
circuits_Alice_unlock.clear()

for i in range(((n*3)//4)+((n*3)//2)):
    Rx_circ = QuantumCircuit(q,c) 
    Rx_circ.rx(2*np.pi*Alice_unlock_rx[i]/360,q[0])
    Ry_circ = QuantumCircuit(q,c) 
    Ry_circ.ry(2*np.pi*Alice_unlock_ry[i]/360,q[0])
    Rz_circ = QuantumCircuit(q,c) 
    Rz_circ.rz(2*np.pi*Alice_unlock_rz[i]/360,q[0])
    #Rz_circ.barrier(q)
    circuits_Alice_unlock.append(Rz_circ+Ry_circ+Rx_circ)
    
print("length of circuits_Alice_unlock : {}".format(len(circuits_Alice_unlock)))
circuits_Alice_unlock[0].draw()

length of circuits_Alice_unlock : 18


In [38]:
# Alice unlocks and measures decoy              

circuits_Alice_unlock_decoy=[]
circuits_Alice_unlock_decoy.clear()


circuits_Alice_unlock_m=[]
circuits_Alice_unlock_m.clear()

for i in range((n*3)//4):
    if preshared_string[i]==0:  # mdm
        circuits_Alice_unlock_m.append(circuits_Alice_unlock[3*i])
        circuits_Alice_unlock_decoy.append(circuits_Alice_unlock[3*i+1])
        circuits_Alice_unlock_m.append(circuits_Alice_unlock[3*i+2])
    elif preshared_string[i]==1:  #dmm
        circuits_Alice_unlock_decoy.append(circuits_Alice_unlock[3*i])
        circuits_Alice_unlock_m.append(circuits_Alice_unlock[3*i+1])
        circuits_Alice_unlock_m.append(circuits_Alice_unlock[3*i+2])
    elif preshared_string[i]==2:  #mmd
        circuits_Alice_unlock_m.append(circuits_Alice_unlock[3*i])
        circuits_Alice_unlock_m.append(circuits_Alice_unlock[3*i+1])
        circuits_Alice_unlock_decoy.append(circuits_Alice_unlock[3*i+2])
    

    
print("length of circuits_Alice_unlock_decoy : {}".format(len(circuits_Alice_unlock_decoy)))
print("length of circuits_Alice_unlock_m : {}".format(len(circuits_Alice_unlock_m)))

circuits_Alice_unlock_decoy[0].draw()
#circuits_Alice_unlock_m[0].draw()

length of circuits_Alice_unlock_decoy : 6
length of circuits_Alice_unlock_m : 12


In [39]:
# X measurement

Alice_X_measurement = QuantumCircuit(q,c) # X basis
Alice_X_measurement.barrier(q)
Alice_X_measurement.h(q[0])
Alice_X_measurement.measure(q[0],c[0])

circuits_Alice_unlock_and_measure_decoy=[]
circuits_Alice_unlock_and_measure_decoy.clear()

for i in range(((n*3)//4)):
    circuits_Alice_unlock_and_measure_decoy.append(circuits_Alice_check_decoy[i]+circuits_Alice_unlock_decoy[i]+Alice_X_measurement)
    

print("length of circuits_Alice_unlock_and_measure_decoy : {}".format(len(circuits_Alice_unlock_and_measure_decoy)))

circuits_Alice_unlock_and_measure_decoy[0].draw()

length of circuits_Alice_unlock_and_measure_decoy : 6


In [40]:
from qiskit import execute, BasicAer
backend_sim = BasicAer.get_backend('qasm_simulator')

job_sim1 = qiskit.execute(circuits_Alice_unlock_and_measure_decoy, backend_sim, shots=1024)
result_sim1 = job_sim1.result()
counts1 = result_sim1.get_counts(circuits_Alice_unlock_and_measure_decoy[0])
print(counts1)

{'00': 1024}


In [41]:
# get result of Alice measuring decoy photons

Alice_measure_decoy_result=[]
Alice_measure_decoy_result.clear()

for i in range(((n*3)//4)):
    Alice_measure_decoy_result.append(int((list(result_sim1.get_counts(circuits_Alice_unlock_and_measure_decoy[i]))[0])[1]))
    
print("Alice_measure_decoy_result = {}".format(Alice_measure_decoy_result))

Alice_measure_decoy_result = [0, 0, 0, 0, 0, 0]


In [43]:
# Check result of decoy photons
right_times_decoy = 0

for i in range(((n*3)//4)):
    if decoy[i]==Alice_measure_decoy_result[i]:
        right_times_decoy+=1
        print("measure {} = origin {}".format(Alice_measure_decoy_result[i],decoy[i]))
    else :
        print("measure {} != origin {}".format(Alice_measure_decoy_result[i],decoy[i]))
        print("Different result has been found.")


print("")
        
error_rate_decoy=((((n*3)//4)-right_times_decoy)/((n*3)//4))
print("error rate = {}%".format(error_rate_decoy*100))

measure 0 = origin 0
measure 0 = origin 0
measure 0 = origin 0
measure 0 = origin 0
measure 0 = origin 0
measure 0 = origin 0

error rate = 0.0%


In [44]:
# Step6 : If no Eve, Alice recovers m 

circuits_Alice_unlock_m_after_check=[]
circuits_Alice_unlock_m_after_check.clear()

for i in range(((n*3)//2)):
    circuits_Alice_unlock_m_after_check.append(circuits_Alice_m_after_check[i] + circuits_Alice_unlock_m[i])
    

print("length of circuits_Alice_unlock_m_after_check : {}".format(len(circuits_Alice_unlock_m_after_check)))

circuits_Alice_unlock_m_after_check[0].draw()

length of circuits_Alice_unlock_m_after_check : 12
